<a href="https://colab.research.google.com/github/Parth-2701/CyFuture_Hackathon/blob/main/LegalAnsweringUsingCUAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

### Unsloth

In [12]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 13500 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Data Prep



In [14]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="my_alpaca_format_for_llama.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model


In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [16]:
for i in range(3):
    print(f"--- Example {i} ---")
    print(dataset[i]["text"])
    print()

--- Example 0 ---
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract

### Input:
Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,244 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.494900
2,1.532800
3,1.696600
4,1.596100
5,1.762500
6,1.527700
7,1.633800
8,1.475400
9,1.413500
10,1.387900


<a name="Inference"></a>
### Inference



In [21]:
context="""EXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  \"Agreement\")  is made by and between Electric City Corp.,  a Delaware  corporation  (\"Company\")  and Electric City of Illinois LLC (\"Distributor\") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company's  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an \"Energy  Saver\"  which may be improved  or  otherwise  changed  from its present composition (the \"Products\").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.\n\n         B. Representations.  As an inducement to the Company to enter into this Agreement,  the  Distributor  has  represented  that  it has or  will  have  the facilities,  personnel,  and financial capability to promote the sale and use of Products.  As an  inducement  to  Distributor  to enter into this  Agreement the Company has  represented  that it has the  facilities,  personnel  and financial capability to have the Products  produced and supplied as needed pursuant to the terms hereof.\n\n         C. The Distributor's  Objectives.  The Distributor  desires to become a distributor  for the Company and to develop  demand for and sell and  distribute Products  solely  for the use within the State of  Illinois,  including  but not limited to public  and  private  entities,  institutions,  corporations,  public schools, park districts,  corrections facilities,  airports,  government housing authorities and other government agencies and facilities (the \"Market\").\n\n         D. The Company's  Appointment.  The Company appoints the Distributor as an  exclusive  distributor  of Products in the Market,  subject to the terms and conditions of this Agreement.\n\n         1.       ESTABLISHMENT OF DISTRIBUTORSHIP\n\n                  1.1      Grant  and   Acceptance.   Company  hereby   appoints                            Distributor as Company's exclusive distributor within                            the Market and grants to  Distributor  the  exclusive                            right  to sell and  distribute  Products  within  the                            Market,   and   Distributor   hereby   accepts   such                            appointment  and such grant,  in accordance  with the                            terms and conditions of this  Agreement.  Distributor                            acknowledges that customers of other  distributors of                            the Products may have sites,  locations or operations                            in  the   Market,   which  will  use  the   Products.                            Distributor  will sell any and all Products  required                            by such  customers in the Market to those  customers.                            Distributor also  acknowledges  that if its customers                            have  sites,  locations  or  operations  outside  the                            Market,   in  the   market   of   another   exclusive                            distributor of the Products,  those customers will be                            required to  purchase  products  from the  applicable                            exclusive  distributor  in  that  market;  otherwise,                            Distributor shall be free to sell to its customers in                            any  market  which  does not have  another  exclusive                            distributor.\n\n                  1.2      License.  The Company  hereby grants the  Distributor                            the right to do business  and use the name  \"Electric                            City of  Illinois\"  or a  similar  variation  thereof                            (collectively   the   \"Names\")  for  use  under  this                            Agreement.  Distributor may file with the appropriate                            state and local authorities assumed name certificates                            as required.  Copies of all documents relating to the                            use of the Names shall be  forwarded  to the Company.                            Upon termination of this Agreement  Distributor shall                            have no further  right to the Names and said  License                            to use the Names shall terminate.  Distributor  shall                            have  no  right  to  sublicense  the  Names  or to do                            business  under any other names without the Company's                            prior  approval in writing.  The parties  acknowledge                            that the  terms  herein  consist  of there  terms for                            Illinois.  At the  request  of  either  party,  a new                            agreement reflecting the terms and conditions of this                            Agreement,  may be executed  for each state or entity                            representing each state.\n\n\n\n\n\n                  1.3      Term.  The term of this  Agreement  shall be ten (10)                            years (the \"Term\")  which shall  commence on the date                            upon which the Company  delivers to  Distributor  the                            last Sample, as defined  hereinafter.  If Distributor                            complies with all of the terms of this Agreement, the                            Agreement  shall be  renewable on an annual basis for                            one (1) year terms for up to  another  ten (10) years                            on the same terms and conditions as set forth herein.                            All renewals of this  Agreement  shall be on the same                            terms and conditions as are set forth herein.\n\n                  1.4      Company's Obligation.  Company shall sell and deliver                            as  provided  in  Section  2.3 of this  Agreement  to                            Distributor  on the  price  terms  set  forth in this                            Agreement  or as  amended  from  time  to  time  such                            quantity of Products as Distributor from time to time                            orders  from  Company.   Company  shall  promote  and                            advertise the Products generally, at its own expense,                            and   shall   furnish   Distributor   copies  at  all                            advertisement and promotional materials.\n\n                  1.5      The Distributor's Obligation. The Distributor, at its                            own expense,  shall promote the distribution,  sales,                            and use of Products in the Market.\n\n                  1.6      Distributor's  Terms  and  Minimum  Expectations.  In                            order  to  maintain  the  exclusive  rights  to sell,                            lease, distribute and service Products in the Market,                            the Distributor must use all commercially  reasonably                            efforts to purchase for sale to  subdistributors  the                            following minimum quantities of the Products from the                            Company:\n\n         On the  commencement of the Term  Distributor will issue to the Company an  irrevocable  letter of credit (\"LC\") in the amount of Five Hundred  Thousand Dollars  ($500,000),  the form of which is  attached  hereto  as  Exhibit  A and incorporated  herein by reference.  The LC shall have a two (2) month term,  and shall be renewed  for five (5)  consecutive  bimonthly  periods.  A minimum of a $250,000.00  purchase  order  must be  received  by Company by the first of each month for a total (12) month  period.  The Company may draw funds from the LC to pay for  Distributor's  purchases,  which are not paid according to the terms in Section 2.7. Prices for the EnergySaver units are\n\n                                    Page -2-\n\nprovided  by the  Company as Exhibit C. The  Company  will be  entitled  to draw against the LC pursuant to the terms of the LC.\n\n                  (A)    375 units in the first Product Year (1999)\n\n                  (B)    750 units in the next succeeding Product Year; (2000)\n\n                  (C)    937 units in the next succeeding Product Year; (2001)\n\n                  (D)    1,171 units in the next succeeding Product Year; (2002)\n\n                  (E)    1,463 units in the next succeeding Product Year; (2003)\n\n                  (F)    1,828 units in the next succeeding Product Year; (2004)\n\n                  (G)    2,285 units in the next succeeding Product Year; (2005)\n\n                  (H)    2,856  unit each in the lat  three  years of                          the initial Term of this  Agreement  and any                          renewals thereof.\n\n         For purposes of this Agreement, a Product Year shall be the twelve (12) month period  following the  commencement  of the initial Term of this Agreement and each twelve  (12)  months  thereafter.  Distributor's  expected  sales shall include the purchase of the Samples as defined hereinafter.\n\n         Sales in excess of the  expected  sales  which are  actually  made in a Product Year may be applied to meet the  expectations for the next Product Year. Any such carry-over from one year to the next Product Year may not be considered in determining  whether there is a carry-over from that next Product Year. Thus, by way of example and not limitation,  if there was an expectation of 50 in year one and 200 for  year two and 60 units  are sold in year one and 195  units  are sold in year two,  the  expectation  for year two will have been met,  but there will be no carry-over to year three. If the  Distributor  shall fail to purchase the minimum number of units in any year, the  Distributor's  exclusive rights to sell and distribute the Product in the Market,  may at Company's sole option, be reevaluated.\n\n         Company  agrees  that  Distributor  shall not be liable or  subject  to reevaluation  for  failure to meet  expectations  due to any  occurrence  beyond Distributor's  reasonable control,  including,  but not limited to, Acts of God, fires,  floods,  wars,  sabotage,  accidents  in shipping,  labor  disturbances,\n\n\n\n\n\nweather conditions,  governmental regulation, lack of Company performance, delay by  Company,  failure of Company to honor  warranties  or  otherwise  materially support the Products.\n\n         The aggregate  units to be sold on an annual basis  described above are for the Illinois distributorship on an aggregate basis.\n\n         1.7      Relationship of Parties.  The relationship between the Company                   and the Distributor\n\n                                    Page -3-\n\n                  is that of vendor and vendee.  This  Agreement does not create                   the  relationship  of principal  and agent between the Company                   and the Distributor for any purpose whatsoever. This Agreement                   shall not be construed as constituting the Distributor and the                   Company as partners, joint venturers, or as creating any other                   form of legal  association or  arrangement  which would impose                   liability  upon one party for the act or omission of the other                   party.  Neither  party is granted any express or implied right                   of  authority  by the other  party to assume or to create  any                   obligation  or  responsibility  on behalf of or in the name of                   the other  party,  or to bind the other party in any manner or                   thing whatsoever.\n\n2.       PURCHASE OF PRODUCTS\n\n         2.1      Orders.  The Distributor shall order Products from the Company                   on a purchase  order form  mutually  acceptable to the Company                   and Distributor and which is consistent with Exhibit B hereto,                   and  which  incorporates  the  terms  and  provisions  of this                   Distributor  Agreement.  The  Distributor  shall  not order or                   purchase Products from any source other than the Company.  All                   orders shall be subject to acceptance and  confirmation by the                   Company.  Distributor  may  cancel an order  that is  properly                   cancelled by  Distributor's  customer,  unless the Company has                   commenced  production  which is in any way customized for that                   customer.  The Distributor  shall annually provide the Company                   with a  non-binding  forecast of orders for  Products  for the                   succeeding 12-month period.\n\n         2.2      Shipment.  The  Company  and  the  Distributor  shall  jointly                   determine  shipment dates.  The Company shall use commercially                   reasonable  efforts to ship  promptly  all orders for Products                   received from the Distributor. In addition to any other remedy                   which this Agreement provides to Distributor  against Company,                   if Company fails to deliver or delays in  delivering  Products                   as were  ordered by  Distributor  within 45 days  after  their                   required  delivery date, and if as a result of such failure or                   delay  Distributor  loses  its  customer's  orders  for  those                   Products,  the number of units which  Distributor  ordered but                   were not timely  delivered to Distributor or to  Distributor's                   customer  will  be  credited  against   Distributor's  minimum                   expectation as specified on Section 1.6 of this Agreement. The                   Company  may refuse to accept a purchase  order on the grounds                   that it cannot meet the delivery schedule therein, and if as a                   result  of  such  failure  or  delay   Distributor  loses  its                   customer's  orders  for those  Products,  the  number of units                   which  Distributor  ordered but were not timely  delivered  to                   Distributor  or to  Distributor's  customer  will be  credited                   against   Distributor's   minimum.   Distributor   shall  make                   reasonable  efforts  to notify  the  Company  of the  proposed                   delivery  schedule before accepting a customer order and shall                   give the  Company  written  notice  of any  customer  purchase                   orders which  imposes  liability for late shipment and neither                   the  Distributor  nor the Company  shall have a liability  for                   consequential  or  liquidated   damages   pertaining  to  late                   delivery unless Company  specifically  acknowledges and agrees                   in  writing  to the  same.  The  Distributor  agrees  that the                   Company  shall not be liable for its failure to perform due to                   any  occurrence  beyond  the  Company's   reasonable  control,                   including, but not limited to, acts of God, fires,\n\n                                    Page -4-\n\n                  floods,  wars,  sabotage,  accidents  in  shipping  beyond the                   Company's  control,  labor strikes other than strikes  against                   the Company itself,  weather conditions or foreign or domestic                   government  regulation  or authority  which  directly  affects                   Company's ability to deliver Product.\n\n         2.3      Delivery.  Other than \"drop ship\"  deliveries,  all deliveries\n\n\n\n\n\n                  made  pursuant to this  Agreement  shall be FOB the  Company's                   facilities  located within the continental  United States by a                   carrier authorized by the Distributor.\n\n         2.4.     Prices.\n\n                  (A)      Prices For Basic  Units.  The prices for  Products in                            the first  Product  Year are  supplied  by Company as                            Exhibit C.\n\n                  (B)      Inflation Price  Adjustment.  The prices set forth in                            Section   2.4(a)  shall  be  subject  to   adjustment                            annually  on the  first  day  of  each  Product  Year                            beginning in the calendar  year 2000 and on the first                            day of each succeeding Product Year for the remainder                            of the Term and all  renewals  of this  Agreement  in                            proportion   to  the  increase  or  decrease  in  the                            Consumer  Price Index (CPI) as compared to the CPI as                            it  existed  on the  first  day of the  Term  of this                            Agreement.  The Company  also  reserves  the right to                            increase  or  decrease  the price  per unit  based on                            Company   wide   changes   in  unit   prices  to  all                            distributors of the Company,  provided however,  that                            any price changes, other than those based on the CPI,                            shall be uniformly applied to all distributors of the                            Products   and  shall   reasonably   applied  to  all                            distributors  of the  Products  and shall  reasonably                            reflect Company's costs of manufacturing the Products                            and/or  market  demand  for  the  Products,  provided                            further  than any increase in price based upon market                            demand   shall   not  be  so  great  as  to   deprive                            Distributor  of  its  normal  and  customary   profit                            margin.  The Company agrees to exercise this right in                            good faith,  and  consider all  circumstances  of the                            Distributor  and the  Company.  The CPI  referred  to                            herein in issued by the Bureau of Labor Statistics of                            the U.S.  Department  of Labor.  Should the Bureau of                            Labor Statistics discontinue  publication of the CPI,                            the parties shall accept comparable statistics on the                            purchasing  power of the consumer dollar as published                            at the time of said  discontinuation  by  responsible                            periodical  or  recognized  authority to be chosen by                            the parties.\n\n         2.5.     Resale Prices.  The  Distributor  may resell  Products at such                   price,  as the  Distributor,  in its  sole  discretion,  shall                   determine.  While the Company has the right to suggest a range                   of  manufacturer's  suggested  retail prices for the Products,                   the  distributor  is not obligated to set retail prices within                   the Company's suggested range of retail prices.\n\n         2.6      Product Returns.\n\n                                    Page -5-\n\n                  (A)      Non-defective Products.  Unless the Company has first                            authorized   or   permitted   the   return   of   any                            non-defective   Products   and  except  as  otherwise                            permitted or required  herein,  the Company shall not                            be   obligated   to  accept  the   return   from  the                            Distributor  of any  non-defective  Products,  nor to                            make  any  exchanges  therefor,  nor  to  credit  the                            Distributor therefor. If Company does not give Annual                            Notice  pursuant to Section  3.1 hereof,  Distributor                            may, within 90 days of  modification,  improvement or                            alteration,  return the Products to the Company.  The                            Company shall not have any obligation with respect to                            Products  after  365  days   following   delivery  to                            Distributor, except as provided herein.\n\n                  (B)      Defective  Products.  In the event of any  damages or                            other  defect in a  Product  which is  discovered  by                            Distributor   within   365   days   of   satisfactory                            installation  of  a  Product  at  Distributor's  or a                            subdistributor's   customer,  the  Distributor  shall                            promptly   report  the  same  to  the   Company   and                            reasonably  demonstrate the defect to the Company. If                            the  Distributor  reasonably  demonstrates  that  the                            Company is responsible for such damage or defect, the                            Company  shall  promptly  deliver  and install at the                            Company's expense,  additional or substitute Products                            to the  subdistributor's  customer without additional                            cost or charge to the Distributor or the customer for                            material,  labor,  shipping,  insurance  or any other                            charge.\n\n         2.7      Payment Terms.  Distributor  shall pay Company  within  thirty\n\n\n\n\n\n                  (30) days of   Distributor's or,  as  the  case  may  be,  the                   end-user's receipt of Products.\n\n         2.8      Company  Cooperation.  The Company  shall  cooperate  with the                   Distributor  in obtaining all necessary  permits and approvals                   to permit  the use of the  Products.  The  Company  shall bear                   responsibility  for any  permits  needed  to  manufacture  the                   Products and  Distributor  shall bear  responsibility  for any                   permits needed to distribute the Products.\n\n3.       PRODUCTS AND WARRANTY\n\n         3.1      Product  Improvements  by the Company.  At the Company's  sole                   discretion,  and  at  any  time,  the  Company  may  give  the                   Distributor at least 90 days advance notice (\"Annual  Notice\")                   of any  modification,  improvement  or  alteration of Products                   (\"New  Products\")  and  development  of new models of Products                   (collectively  with  \"New  Products\",   \"Improved  Products\").                   Except for the  Improved  Products  for which the  Distributor                   receives the Annual  Notice,  the Company  shall sell Improved                   Products  to   Distributor   only  with  the  consent  of  the                   Distributor.  Any  Improved  Products  shall be subject to the                   provisions  of  this  Agreement.   Old  Products  will  remain                   available unless\n\n                                    Page -6-\n\n                  Improved Products perform at the same or better levels and are                   offered at reasonably  similar prices or at prices  increases,                   which reasonably reflect improvements in performance.\n\n         3.2      Product Improvements by the Distributor. The Distributor shall                   disclose  to  the  Company  any   modifications   to  Products                   requested  by  end-users  or  other   proposals   for  Product                   improvement from end-users or the Distributor,  but shall have                   no right to make modifications without Distributor's consent.\n\n         3.3      Warranty.  Company shall at all times make reasonable  efforts                   to  maintain  quality  control  and  to  deliver  Products  to                   Distributor  which,  when received by Distributor,  or, as the                   case  may  be,  the  end-user,  are  properly  and  adequately                   packaged   and   contained,   fully   assembled   (except  for                   miscellaneous  components  which may be shipped  separately to                   prevent damage in transit),  fully functional and otherwise in                   conformance  with the  warranties  set forth  herein.  Company                   warrants  that the Products  will be  designed,  manufactured,                   constructed,  assembled and packaged in a  workmanlike  manner                   and that such Products  shall be fully  functional and fit for                   their intended  purposes.  Company  further  warrants that the                   Products sold hereunder  shall be free from defects in design,                   materials and  workmanship  for a period of  twenty-four  (24)                   months after delivery to Distributor's  end-user.  The Company                   shall not be liable for defective Products, except as provided                   in this  Agreement.  The Distributor at all times shall comply                   with all  requirements of the  Magnuson-Moss  Warranty-Federal                   Trade Commission Improvement Act and similar federal and state                   laws and regulations.\n\n         3.4      Warranty Work. If, within the twenty-four  (24) month warranty                   period set forth above,  Company  received from Distributor or                   any of  Distributor's  end-user's  a notice  which may be oral                   notice  confirmed in writing)  that any of the  Products  sold                   hereunder do not meet the Warranties  specified above, Company                   shall  thereupon  correct  each such defect by  providing  the                   necessary repairs,  and/or replacement parts, or if necessary,                   Products.  Company shall promptly respond to any timely notice                   of defect.  Unless otherwise expressly agreed to in writing by                   Distributor or Distributor's and-user,  Company shall bear the                   reasonable  expense  of  all  labor,  materials  and  shipping                   expended  or used in  connection  with the  correction  of any                   defects in the Products  occasioned by the  non-conformance of                   the  Product  with  Company's  warranty  as set forth  herein.                   Company  shall be  entitled  to  dispute  whether a Product is                   defective. In the event that Company is unable or unwilling to                   promptly  perform any warranty work without  reasonable  cause                   and following  full and fair  opportunity  to do so, or in the                   event of the necessity  for  emergency  repairs of a defective                   Product  for  which  there  is no  reasonable  possibility  of                   performance by Company,  Distributor may perform such warranty                   work or hire a third party to perform such  warranty  work and                   the reasonable cost thereof shall be paid by Company.\n\n                                    Page -7-\n\n\n\n\n\n         3.5      Service of  Products  in  Territory.  Within  thirty (30) days                   after the  execution  of this  Agreement,  the Company and the                   Distributor shall mutually agree upon a reasonable schedule of                   charges for after market  parts and  services  provided by the                   Company  or  the  Distributor  so  that  such  charges  do not                   adversely affect the marketability of the Products.\n\n         3.6.     Non-Disclosure  of  Confidential  Information.   None  of  the                   parties   hereto  nor  their   associated   or  affiliated  or                   affiliated  companies  shall during the term of this Agreement                   or thereafter disclose any confidential  information  obtained                   or acquired by them in  connection  with the  Products and the                   business of the other,  including,  without limitation,  trade                   secrets, business techniques, technical information,  customer                   and potential customer lists,  marketing data and information,                   prices,  improvements  to the  Products  in various  stages of                   development,  processes,  or  other  confidential  information                   relating to the Products or the  business of the  Distributor,                   except that either  party shall be  permitted  to disclose (x)                   all or portions of such confidential information on a strictly                   need-to-know  basis to the  extent  required  by an order of a                   court of competent jurisdiction or by the order or demand of a                   regulatory body having  jurisdiction  over one or both parties                   and (y) any of such confidential  information that is the sole                   property  of the  party  making  the  disclosure  and does not                   include  any  information   owned  by  the  other  party.  The                   Distributor  shall not  disclose  this  agreement  except upon                   consent of Company. Confidential information shall not include                   information which:\n\n                  (A)      Is or becomes  generally  available  to the party who                            desires  to  disclose   such   information   (or  its                            associated  or affiliated  companies) (a  \"Disclosing                            Party\")  other  than as a result  of a breach of this                            Agreement or some other unlawful means;\n\n                  (B)      Becomes  available  to  the  Disclosing  Party  on  a                            non-confidential  basis  from a  third  party  who is                            under no confidentiality or nondisclosure  obligation                            with respect to such information; or\n\n                  (C)      Was   known   to   the   Disclosing    Party   on   a                            non-confidential   basis  prior  to  the   disclosure                            thereof to such  disclosing  Party by a party to this                            Agreement.\n\n4.       DURATION AND TERMINATION\n\n         4.1      Duration.   Unless  earlier   terminated   otherwise  provided                   therein,  this  Agreement,  subject to the  commencement  date                   established  in Section 1.3,  shall be effective  immediately.                   Distributor  shall submit written  reports to the Company each                   quarter during the first year of the Term,  commencing  ninety                   (90) days after  execution of this  Agreement,  describing its                   efforts,  the potential  customers it has  approached  and the                   status of its efforts.\n\n         4.2      Termination  for  Cause.   Either  party  may  terminate  this                   Agreement upon 30 days\n\n                                    Page -8-\n\n                  prior written  notice to the other upon the  occurrence of any                   of the following events: (A) the Distributor's failure to make                   full and  prompt  payment  to the  Company of all sums due and                   owing to it; (b) either party's  default in the performance of                   any  of  the   material,   terms,   conditions,   obligations,                   undertakings,  covenants or  liabilities  set forth herein and                   such  default is not cured  within a  commercially  reasonable                   time  after  the  defaulting  party has been  notified  of the                   default  by the  other  party and (c) as  otherwise  expressly                   provided  herein.  In  the  event  either  party  (a)  becomes                   adjudicated insolvent,  (b) discontinues its business, (c) has                   voluntary of  involuntary  bankruptcy  proceedings  instituted                   against  it, or (d) makes an  assignment  for the  benefit  of                   creditors, the other party shall be entitled to terminate this                   Agreement effective immediately upon written notice.\n\n         4.3      Accrued  Obligations.  In the event that either Distributor or                   Company fails to comply with the terms of this Agreement, both                   Distributor and Company acknowledge and agree that in addition                   to any claim for damages  either  party may have  arising from                   the  default of the  other,  they shall have the right to seek                   equitable  relief  by way of a  temporary  restraining  order,                   preliminary  injunction,  permanent  injunction and such other                   equitable  relief as may be appropriate.  In the event a party                   seeks the equitable relief of a temporary  restraining  order,                   preliminary   injunction,   permanent  injunction,   mandatory\n\n\n\n\n\n                  injunction or specific  performance  both parties  acknowledge                   that they shall not be required to demonstrate  the absence of                   an adequate remedy at law, and neither party shall be required                   to post  bond  as a  precondition  to  obtaining  a  temporary                   restraining order or preliminary  injunction.  The termination                   of this  Agreement  shall not relieve either party hereto from                   obligations  which have occurred pursuant to the provisions of                   this Agreement prior to its termination,  nor shall it release                   either  party  hereto  from any  obligations  which  have been                   incurred  as a  result  of  operations  conducted  under  this                   Agreement.\n\n         4.4      Repurchase of Products.  Upon the expiration or termination of                   this  Agreement,  pursuant to Section  4.1 or 4.2 hereof,  the                   Company may, at its option to be  exercised  within 30 days of                   the date of the termination of this Agreement, and in its sole                   discretion,  repurchase  any Products in the possession of the                   Distributor  at the net invoice price paid by the  Distributor                   to  the  Company  less  any  applicable  special   allowances,                   discounts, shipping or allowances for cooperative advertising.                   If Company  terminates  the  Agreement  without  cause and for                   reasons other than  Distributor's  failure to meet its minimum                   expectations;   it  shall   repurchase  from  Distributor  any                   unopened  Product,  and shall bear all shipping,  handling and                   related  costs  notwithstanding  any other  remedies  to which                   Distributor  may be  entitled.  On  demand  and  tender of the                   repurchase  price,  the  Distributor  shall  be  obligated  to                   deliver such Products to the Company. The Company reserves the                   right to reject any Products  that are not factory  sealed and                   in new and unused  condition.  Repurchased  Products  shall be                   shipped at the Company's  expense,  and the Company may offset                   any indebtedness of the Distributor to the Company against the                   repurchase  price of such  Products.  Following  expiration or                   termination of this Agreement, the Distributor may continue to                   sell any Products in the Market which are in its inventory\n\n                                    Page -9-\n\n                  and which the Company has not repurchased.\n\n5.       REPRESENTATIONS AND WARRANTIES AND OTHER MATTERS\n\n         5.1      Representations and Warranties of Company.\n\n                  (A)      The  Company  represents  that,  to the  best  of its                            knowledge,  Products are in compliance with all laws,                            and  that  the  Products  will  not be  hazardous  or                            dangerous  when  used  for  their  intended  purpose.                            Products  do not  cause  harmful  emissions  or other                            environmental  hazards and Products do not violate or                            infringe any patents, copyrights, trademarks or other                            rights  of  nay  third  party(ies).  Company  further                            represents   and  warrants  that  its  Products  will                            perform as  advertised  and  promoted by the Company,                            and will be approved  or  certified  by  Underwriters                            Laboratory.\n\n                  (B)      The  Company  will  make   available  to  Distributor                            comprehensive technical support for the first Product                            Year.  Distributor will have direct access to (a) the                            Company's engineering  consultants and (b) the patent                            holder's technicians.  Company's representatives will                            make themselves available three days per month in the                            first   Product   Year  to  consult  with  and  train                            Distributor.  All costs and expenses  associated with                            the  services  provided  to  Distributor   hereunder,                            including travel, lodging,  engineering  consultants'                            fees and employee time will be paid by Distributor.\n\n                  (C)      Company  will  timely  furnish  all of  Distributor's                            requirements for Products within the Market, provided                            it  is  given   adequate   notice  of   Distributor's                            requirements  and a  full  and  fair  opportunity  to                            fulfill the same.\n\n         5.2      Representations and Warranties of Distributor.\n\n                  (A)      Distributor   shall  be  entirely   responsible   for                            learning,   understanding   and  training  about  the                            Products,  the costs of advertising and promoting the                            Products  in the  Market  through  the  Term  of this                            Agreement.  Distributor  shall  not  issue,  print or                            disseminate any information about the Products in the                            first  Product  Year  without  the  express   written                            consent of the Company.\n\n                  (B)      Distributor  will  not  engage  the  services  of any                            engineering  or  consulting  firm without the express\n\n\n\n\n\n                           written consent of the Company.\n\n         5.3      Indemnification.  Company and Distributor  agree to indemnify,                   defend  and hold each other  harmless  from any and all suits,                   claims, obligations, liabilities, damages, losses and the like                   (including  attorneys'  fees and costs) relating to or arising                   out  of:  (A)  any  breach  of any  material  representations,                   warranties,  covenants,  obligations,  agreements or duties in                   connection with this  Agreement;  (b) any negligence or fault;                   (c) any violation by either of them of the patent,  copyright,                   trademark or other\n\n                                    Page -10-\n\n                  intellectual  property  rights of third parties.  In addition,                   Company   agrees  to  indemnify,   defend  and  hold  harmless                   Distributor from and against all suits,  claims,  obligations,                   liabilities,   damages,   losses   and  the  like   (including                   attorneys'  fees  and  costs)  arising  out of or  related  to                   Company's manufacture or design of the Products, provided that                   Distributor  is not at fault in connection  with the same, and                   Distributor  agrees to  indemnify,  defend  and hold  harmless                   Company  from and  against  all  suits,  claims,  obligations,                   liabilities,   damages,  losses  and  the  like  (including  a                   attorneys'  fees  and  costs)  arising  out of or  related  to                   Distributor's  sales,   marketing  practices  or  unauthorized                   Product  alteration  (provided that Company is not at fault in                   connection with same).\n\n         5.4      Product Liability  Insurance.  Company will carry a reasonable                   amount of product  liability  insurance  through a  reasonably                   acceptable  products liability insurance company and will name                   the  Distributor  as an additional  insured under that policy.                   Company  will make  reasonable  efforts  to  procure a policy,                   which is non-cancelable, except upon thirty (30) days, advance                   notice to the Distributor.\n\n         5.5      No License. The Distributor acknowledges and agrees the except                   as provided by Section 1.2 of this  Agreement,  this Agreement                   will not be construed as granting by implication,  estopped or                   otherwise  any license or other  right of use with  respect to                   any present or future patent, copyright, trademark, trade name                   or other proprietary right owned by or licensed to the Company                   or any of its affiliates.\n\n         5.6      No Action to Invalidate. During the Term of this Agreement and                   for three  years  thereafter,  the  Distributor  (on behalf of                   itself and each of its affiliates) agrees not to commence,  or                   provide any  information to or otherwise  assist any person or                   entity in  connection  with,  any suit,  action or  proceeding                   contesting the ownership,  validity or  enforceability  of any                   patent,  copyright,  trademark,  trade name or other propriety                   right owned by or licensed to the Company,  whether  currently                   existing or hereinafter invented, developed or acquired unless                   required to by court order.  The Distributor  agrees to inform                   the Company  promptly  and  cooperate  with the Company in the                   event the  Distributor  obtains  knowledge  of any such  suit,                   action  or   proceeding   which  has  been   initiated  or  is                   contemplated by any other person or entity.\n\n         5.7      Nonsolicitation.\n\n                  (A)      During the Term of this Agreement and for a period of                            twelve (12) months  thereafter,  the  Distributor (on                            behalf of itself,  each of its affiliates and each of                            their respective representatives) agrees that it will                            not  directly  or  indirectly  solicit  or  hire  any                            executive,  managerial  or technical  employee of the                            Company or any of its affiliates.\n\n                  (B)      Distributor further agrees that it will not interfere                            with or  otherwise  disrupt  the  business  relations                            between the Company or nay of its  affiliates and any                            of their current or prospective customers,  suppliers                            or distributors, during the\n\n                                    Page -11-\n\n                           Term of the  Agreement  and for a period of  eighteen\n\n\n\n\n\n                           (18) months thereafter,  nor will Distributor solicit                            any  customer  or  potential  customer  of Company to                            purchase a competitive product during that period.\n\n         5.8.     Nonpublic Information. The Distributor acknowledges that is it                   aware that the  securities  laws  prohibit  any person who has                   material, non-public information concerning the Company or the                   matters  which  are  the  subject  of  this   Agreement   from                   purchasing  or selling  securities of the Company (or options,                   warrants and rights relating  thereto) and from  communicating                   such  information to any other person under  circumstances  in                   which it is reasonably  foreseeable that such person is likely                   to purchase or sell such securities.\n\n6.       INTERPRETATION AND ENFORCEMENT\n\n         6.1      Assignment.  No  assignment  of this  Agreement  or any  right                   accruing  hereunder  shall be made by the Distributor in whole                   or in part,  without the prior written consent of the Company,                   which  consent  shall  not  be  unreasonably  withheld.  As  a                   condition  to  obtaining   such   consent,   the  Assignee  of                   Distributor's  interest hereunder shall first agree in writing                   in form and  substance  satisfactory  to the Company,  that is                   shall  assume  and  be  liable  for  the  performance  of  all                   obligations imposes by this Agreement on Distributor,  whether                   such obligations have then accrued are owing, or are yet to be                   performed, and shall demonstrate that is has the economic, and                   with  approval  of the  assignment,  the legal  capability  to                   perform  all  of the  obligations  of  Distributor  hereunder.                   Company  may  assign its  interest  in this  agreement  to any                   person or entity  which has  authority  to  fulfill  Company's                   obligations  hereunder  and which has the economic  ability to                   perform its  obligations  hereunder.  Upon the assignment of a                   party's  interest and rights in this  Agreement  the assigning                   party shall be relieved of all further  obligations imposed by                   this Agreement.          6.2      Nonwaiver of Rights. Failure of either party to enforce any of                   the  provisions  of this  Agreement or any rights with respect                   thereto or failure  to  exercise  any  election  provided  for                   herein shall in no way be a waiver of such provisions,  rights                   or  elections  or in any  way  affect  the  validity  of  this                   Agreement. The failure of either party to exercise any of said                   provisions,   rights  or  elections   shall  not  preclude  or                   prejudice  such party from later  enforcing or exercising  the                   same or any other provisions, rights or elections which is may                   have under this Agreement.\n\n         6.3      Invalid  Provisions.  If any terms,  provision,  covenant,  or                   condition  of this  Agreement  is held by a court of competent                   jurisdiction  to  be  invalid,  void,  or  unenforceable,  the                   remainder  of the  provisions  shall  remain in full force and                   effect  and  shall  in  no  way  be   affected,   impaired  or                   invalidated.\n\n         6.4      Notices.  Any  notice  or  other  communication   required  or                   permitted hereunder shall be in writing and shall be delivered                   personally,   telegraphed,   telexed,  or  sent  by  facsimile                   transmission or sent by certified, registered or express mail,                   postage prepaid. Any such notice shall be deemed given when so                   delivered personally,\n\n                                    Page -12-\n\n                  telegraphed,  telexed or sent by facsimile transmission or, if                   mailed, two (2) business days after the date of deposit in the                   United   States  mail,  by  certified   mail  return   receipt                   requested, as follows:\n\n                  If to the Distributor to:                   Electric City of Illinois L.L.C.                   8628 Oketo Avenue                   Bridgeview, IL  60455                   Facsimile No.  (708) 598-4671                   Attn:  Jim Stumpe\n\n                  With a copy to:                   Thomas V.  McCauley                   200 W.  Adams, Suite 2500                   Chicago, IL  60606                   Facsimile No.  (312) 346-9316\n\n                  If to Company to:                   Electric City Corp.                   1280 Lanmeier Rd.                   Elk Grove Village, IL  60007                   Attn:  Joseph Marino, President\n\n                  With a copy to:                   Kwaitt & Ruben, Ltd.                   211 Waukegan Road                   Suite 300\n\n\n\n\n\n                  Northfield, Illinois  60093                   Attn:    Philip E.  Ruben, Esq.\n\n6.5      Entire Agreement.  This Agreement,  together with all exhibits attached          hereto which are hereby  incorporated by reference,  supersedes any and          all other  agreements,  either  oral or  written,  between  the parties          hereto with  respect to the subject  matter  hereof and contains of the          covenants  and  agreements  between  the parties  with  respect to said          matter. This Agreement may not be altered, amended or modified,  except          by written instrument signed by the parties hereto.\n\n6.6      Sample Products.  Company will,  during the Term of this Agreement (and          any renewal term), provide Distributor,  at Distributor's cost pursuant          to the  terms of this  Agreement,  with  five  (5)  sample  units  (the          \"Sample\" or  \"Samples\")  for use by  Distributor  in  promoting  sales.          Distributor shall use the Samples for purposes of permitting  potential          customers  to use the Products in the field.  The Samples  purchased by          Distributor hereunder shall count toward the minimum expectations under          this Agreement.\n\n6.7      Time of the Essence.  Time is of the essence of this Agreement.\n\n                                    Page -13-\n\n6.8      Force Majeure.  Neither party to this Agreement  shall be liable to the          other  party,  nor shall be subject to  injunctive  relief by the other          party if that party's  performance of its duties or  obligations  under          this  Agreement  is the  consequence  of Force  Majeure  as  defined in          Section 2.2 hereunder.\n\n6.9      Governing Law.  This Agreement is to be construed according to the laws          of the State of Illinois.\n\n7.       NEW PRODUCTS\n\n         7.1      Right of Option.  Should Company  introduce  other products or                   devices as contemplated by recital paragraph \"A\",  Distributor                   shall  have  the  option  of  becoming   Company's   exclusive                   distributor  of such  other  Products  or  devices  within the                   Market.\n\n         7.2      Exercise of Option.  Distributor  shall exercise its option to                   become  exclusive  Distributor of other Products or devices by                   serving  written  notification  on Company of its  election to                   become  exclusive  distributor  within  thirty  (30) days upon                   which  Company  informed  Distributor  in writing of Company's                   intention  to  introduce   other   Products  or  devices.   If                   Distributor  does not exercise its option as herein  provided,                   Company may  distribute  the other  Products or devices within                   the Market itself or through other distributors.\n\n         7.3      Other  Agreements.  The terms  pursuant  to which  such  other                   Products  or devices  shall be sold by Company to  Distributor                   shall  be  determined  by  a  separate  agreement,   but  such                   agreement   shall  be   essentially  on  the  same  terms  and                   conditions as herein provided,  understanding  that such terms                   as price,  quotas,  and  length  of the term of the  agreement                   shall be  reasonably  adjusted  to  reflect  the nature of the                   other Product or device which is the subject of the agreement.\n\n                  In witness whereof the parties have executed this Agreement as of the date first abovementioned.\n\n      Electric City Corp.                       Electric City of Illinois L.L.C.\n\nBy:   /s/Joseph Marino                    By:   Jim Stump       -------------------                       -------------------------------       President\n\n                                    Page -14-
"""

In [22]:
question="""Highlight the parts (if any) of this contract related to \"Document Name\" that should be reviewed by a lawyer. Details: The name of the contract
"""
answer="DISTRIBUTOR AGREEMENT"

In [23]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question, # instruction
        context, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nHighlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract\n\n\n### Input:\nEXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company\'s  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may enga

In [25]:
question="""Highlight the parts (if any) of this contract related to \"Agreement Date\" that should be reviewed by a lawyer. Details: The date of the contract."
"""
answer = "7th day of September, 1999"

In [26]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question, # instruction
        context, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nHighlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract."\n\n\n### Input:\nEXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company\'s  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may e